In [1]:
import pandas as pd
import numpy as np

#### Description

In this project, you will practice techniques of Clustering under unsupervised machine learning using python with the Cambridge Crime Data dataset. The data contains a list of reported crimes in the annual crime report by the Cambridge police department from the year 2009 to 2016. The aim of the project is to divide the crimes into groups/clusters according to the crime type or crime column in the dataset and the neighborhood. The projects tests and strengths your understanding of the following previously discussed classes’ topics:

- K-Means Clustering.
- Hierarchical Clustering.

In [4]:
df = pd.read_csv("Data/Cambridge Crime Data 2009-2016.csv")
df.head(10)

,File Number,Date of Report,Crime Date Time,Crime,Reporting Area,Neighborhood,Location
0,2009-00002,01/01/2009 12:39:00 AM,1/1/09 0:39,Simple Assault,504.0,Cambridgeport,"400 Massachusetts Avenue, Cambridge, MA"
1,2009-00003,01/01/2009 01:34:00 AM,1/1/09 1:34,Simple Assault,610.0,Mid-Cambridge,"200 HAMPSHIRE STREET, Cambridge, MA"
2,2009-00004,01/01/2009 01:43:00 AM,01/01/2009 02:20 - 02:35,Aggravated Assault,708.0,Riverside,"DUNSTER STREET & MOUNT AUBURN STREET, Cambridg..."
3,2009-00005,01/01/2009 02:27:00 AM,01/01/2009 02:20 - 02:45,Simple Assault,105.0,East Cambridge,"100 SCIARAPPA STREET, Cambridge, MA"
4,2009-00006,01/01/2009 02:34:00 AM,01/01/2009 02:15 - 02:35,Disorderly,708.0,Riverside,"1200 Massachusetts Avenue, Cambridge, MA"
5,2009-00007,01/01/2009 02:45:00 AM,1/1/09 2:45,Missing Person,507.0,Cambridgeport,"0 WOODROW WILSON COURT, Cambridge, MA"
6,2009-00008,01/01/2009 02:37:00 AM,1/1/09 2:37,Mal. Dest. Property,1108.0,North Cambridge,"100 MONTGOMERY STREET, Cambridge, MA"
7,2009-00009,01/01/2009 10:42:00 AM,01/01/2009 10:30 - 10:42,Simple Assault,411.0,Area 4,"600 Massachusetts Avenue, Cambridge, MA"
8,2009-00010,01/01/2009 10:47:00 AM,01/01/2009 10:35 - 10:47,Shoplifting,1201.0,Highlands,"200 ALEWIFE BROOK PKY, Cambridge, MA"
9,2009-00011,01/01/2009 12:37:00 PM,12/31/2008 10:00 - 01/01/2009 10:00,Hit and Run,510.0,Cambridgeport,"FAIRMONT Avenue & Fairmont STREET, Cambridge, MA"
